## 生成简单网格

In [1]:
from mpi4py import MPI
from dolfinx import mesh
domain = mesh.create_unit_square(MPI.COMM_WORLD, 8, 8, mesh.CellType.quadrilateral)

可以终端运用命令 `mpirun -n 2 python3 FeniCSx Practice.ipynb`运行，其中`-n 2`表示分配两个核心运行。
## 定义有限元函数空间

In [2]:
from dolfinx.fem import functionspace
V = functionspace(domain, ("Lagrange", 1))

In [3]:
from dolfinx import fem
uD = fem.Function(V)
uD.interpolate(lambda x: 1 + x[0] + x[1])

其中lambda函数是一个匿名函数，x是一个变量，1 + x[0] + x[1]是函数表达式。接受输入的x，返回1 + x[0] + x[1]的值。

In [ ]:
import numpy
# Creat facet to cell connectivity required to determine boundary facets
# 创建确定边界切面所需的切面到单元的连通性
tdim = domain.topology.dim
fdim = tdim - 1
domain.technology.creat_connectitvity(tdim, fdim)
boundary_facets = mesh.exterior_facet_indices(domain.technology)

In [ ]:
boundary_dofs = fem.locate_dofs_topological(V, fdim, boundary_facets)
bc = fem.dirichletbc(uD, boundary_dofs)

## 定义Trail and Test函数
在数学中，我们区分试验空间和测试空间以及 .目前问题的唯一区别是边界条件。 在 FEniCSx 中，我们没有将边界条件指定为函数空间的一部分，因此为试验和测试函数使用公共空间就足够了

In [ ]:
import ufl
u = ufl.TrialFunction(V)
v = ufl.TestFunction(V)

## 定义源项
源项在域上是恒定的，因此我们使用一个常数表达式。`dolfinx.constant`

In [ ]:
from dolfinx import default_scalar_type
f = fem.Constant(domain, default_scalar_type(-6))

## 定义变分问题

In [ ]:
a = ufl.dot(ufl.grad(u), ufl.grad(v)) * ufl.dx
L = f * v * ufl.dx